In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
DAT_FILE = r"C:\Users\shrey\Documents\Projects\pokerai\build\preflop_dist_10000.dat"

TAG_BITS = 64
HIST_SIZE = 50
HIST_BITS = 16

def read_data(filename):
    rv = {}
    with open(filename, "rb") as f:
        while True:
            tag_bytes = f.read(TAG_BITS // 8)
            if not tag_bytes:
                break
            tag = int.from_bytes(tag_bytes, byteorder="little", signed=True)

            vals = []

            for hist_index in range(HIST_SIZE):
                hist_bytes = f.read(HIST_BITS // 8)
                hist_val = int.from_bytes(hist_bytes, byteorder="little", signed=False)
                vals.append(hist_val)

            rv[tag] = vals

    return rv

def dict_to_matrix(d):
    keys = list(range(len(d)))
    matrix = np.zeros((len(d), HIST_SIZE))

    for key in keys:
        matrix[key, :] = d[key]

    return matrix

dists_dict = read_data(DAT_FILE)

In [3]:
dists = dict_to_matrix(dists_dict)
dists.shape

(169, 50)

In [4]:
dists_dict[51]

[351,
 139,
 115,
 188,
 448,
 181,
 561,
 393,
 363,
 429,
 497,
 567,
 577,
 274,
 349,
 207,
 200,
 89,
 102,
 97,
 55,
 42,
 43,
 57,
 75,
 73,
 115,
 102,
 118,
 143,
 252,
 227,
 312,
 165,
 239,
 181,
 171,
 123,
 168,
 158,
 147,
 154,
 115,
 142,
 92,
 151,
 91,
 102,
 18,
 42]